In [186]:
! pip3 install pymysql

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import csv
import pymysql as sql 

def convert_txt_csv(filename):
    final = []
    with open(filename) as f1:
        data = csv.reader(f1)
        gene = next(data)[0].split()[0]
        gene_id = next(data)[0].split()[2]
        #gene = gene[0].split()[0]         #print(gene)
        #gene_id = gene_id[0].split()[2]   #print(gene_id)
    
        check = False
        exon_no = 0
    
        for line in data:
            if len(line) > 0  and 'NM' in line[0]:
                check = True
                trans_id = line[0].split()[4]
                exon_no = 0
                #print(trans_id)
            if check:
                if len(line) > 0 and '-' in line[0]:
                    stend = line[0].split()[0]
                    start = stend.split('-')[0]
                    end = stend.split('-')[1]
                    if end != '' and start != '':
                        exon_no += 1
                        size = int(start) - int(end)
                        final.append([gene, gene_id, trans_id,exon_no, start, end, size])
                        #print(size)
                    #print(start)
                    #print(end)
    #print(final)
    print("conversion done")
    with open('ETL.csv','w',newline='') as f1:
        cs_wr = csv.writer(f1)
        cs_wr.writerow(['Gene', 'Gene_ID', 'Trans_ID','Exon_no', 'Start', 'End', 'Size'])
        cs_wr.writerows(final)


def create_database(db_name):
    connection = sql.connect(host = 'localhost',
                        user = 'root',
                        port = 3306,
                        password = 'shansk@09')

    db = connection.cursor()

    db.execute("create database if not exists %s"%(db_name))
    connection.commit()

    db.close()
    connection.close()

def create_table(db_name,tb_name,column_list):
    
    tb_col = ','.join(["%s varchar(250)"%(i) for i in column_list])
    connection = sql.connect(host = 'localhost',
                        user = 'root',
                        port = 3306,
                        password = 'shansk@09',
                        database = db_name)

    db = connection.cursor()
    db.execute("create table if not exists %s (%s)"%(tb_name,tb_col))
    connection.commit()

    db.close()
    connection.close()
    
def upload_data(db_name,tb_name,col_list,data_list):
    col_nam = ','.join(col_list)
    vals = ','.join(["'%s'"%(i) for i in data_list])
    connection = sql.connect(host = 'localhost',
                        user = 'root',
                        port = 3306,
                        password = 'shansk@09',
                        database = db_name)

    db = connection.cursor()
    db.execute("insert into %s (%s) values (%s)"%(tb_name,col_nam,vals))
    connection.commit()

    db.close()
    connection.close()
    

if __name__ =="__main__":  
    txtfilename = "gene .txt"
    db_name = "DS_mini_project"
    fln = "gene.csv"
    tb_name = fln.replace(".csv","")
    print(db_name,tb_name)
    
    convert_txt_csv(txtfilename)
    create_database(db_name)
    
    with open("ETL.csv") as f1:
        c_read = csv.reader(f1)
        hd = next(c_read)
        #print(hd)
        create_table(db_name,tb_name,hd)
        for line in c_read:
            #print(line)
             upload_data(db_name,tb_name,hd,line)
        
    print("Done")
    


DS_mini_project gene
conversion done
Done
